In [ ]:
from google.colab import files
upload = files.upload()

Saving tcpSyn-APAddos.zip to tcpSyn-APAddos.zip


In [ ]:
! unzip "tcpSyn-APAddos".zip

Archive:  tcpSyn-APAddos.zip
  inflating: proc-model2-tcp-syn.csv  
  inflating: model2-apa-ddos.csv     


In [ ]:
import csv
import tensorflow as tf
import numpy as np
import ipaddress

In [ ]:
datafile = open('proc-model2-tcp-syn.csv','r')
csvr = csv.reader(datafile)
next(csvr)
data = []
label = []
for row in csvr:
  if row[0] == '':
    continue
  row[0] = int(ipaddress.ip_address(row[0]))
  row[1] = int(ipaddress.ip_address(row[1]))
  row[2] = int(row[2])
  row[3] = int(row[3])
  row[4] = int(row[4])
  row[5] = int(row[5])
  row[6] = int(row[6])
  row[7] = int(row[7])
  data.append(row[0:-1])
  label.append(row[-1])

In [ ]:
for i,r in enumerate(label):
  if r == 'benign':
    label[i] = 0
  else:
    label[i] = 1

In [ ]:
train_data = np.asarray(data[0 : int(len(data) * 0.8)])
valid_data = np.asarray(data[int(len(data) * 0.8) : int(len(data) * 0.9)])
test_data = np.asarray(data[int(len(data) * 0.9) : ])

train_label = np.asarray(label[0 : int(len(label) * 0.8)])
valid_label = np.asarray(label[int(len(label) * 0.8) : int(len(label) * 0.9)])
test_label = np.asarray(label[int(len(label) * 0.9) : ])

train_label = tf.keras.utils.to_categorical(train_label, 2)
valid_label = tf.keras.utils.to_categorical(valid_label, 2)
test_label = tf.keras.utils.to_categorical(test_label, 2)

In [ ]:
input_size = len(train_data[0])
output_size = len(set(label))
hidden_layer_size = 75
ACT_FN = 'relu'

model = tf.keras.Sequential([
          tf.keras.layers.Dense(30, input_dim=input_size, kernel_initializer='uniform', activation='relu'), # input layer

          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 1st hidden layer
          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 2nd hidden layer
          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 3nd hidden layer
          tf.keras.layers.Dense(hidden_layer_size, activation=ACT_FN), # 4nd hidden layer
          tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 30)                270       
                                                                 
 dense_25 (Dense)            (None, 75)                2325      
                                                                 
 dense_26 (Dense)            (None, 75)                5700      
                                                                 
 dense_27 (Dense)            (None, 75)                5700      
                                                                 
 dense_28 (Dense)            (None, 75)                5700      
                                                                 
 dense_29 (Dense)            (None, 2)                 152       
                                                                 
Total params: 19,847
Trainable params: 19,847
Non-trai

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import TensorBoard
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)
max_epochs = 2
BATCH_SIZE = 2

In [ ]:
model.fit(train_data,
          train_label,
          # batch_size=BATCH_SIZE,
          epochs=max_epochs,
          # callbacks=tb,
          validation_data=(valid_data, valid_label),
          validation_steps=10,
          verbose=1
          )

Epoch 1/2
38383/38383 [==============================] - 105s 3ms/step - loss: 499.2160 - accuracy: 0.9037 - val_loss: 0.1082 - val_accuracy: 1.0000
Epoch 2/2
38383/38383 [==============================] - 111s 3ms/step - loss: 17.6420 - accuracy: 0.9062 - val_loss: 0.0913 - val_accuracy: 1.0000


In [ ]:
test_loss, test_accuracy = model.evaluate(test_data, test_label)

4798/4798 [==============================] - 8s 2ms/step - loss: 0.0851 - accuracy: 1.0000


In [ ]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 100.00%


In [ ]:
# 
#  TESTING MODEL ON AN ENTIRELY NEW DATASET
#

In [ ]:
datafile2 = open('model2-apa-ddos.csv','r')
csvr2 = csv.reader(datafile2)
next(csvr2)
data2 = []
label2 = []
for row in csvr2:
  if row[0] == '':
    continue
  row[0] = int(ipaddress.ip_address(row[0]))
  row[1] = int(ipaddress.ip_address(row[1]))
  row[2] = int(row[2])
  row[3] = int(row[3])
  row[4] = int(row[4])
  row[5] = int(row[5])
  row[6] = int(row[6])
  row[7] = int(row[7])
  data2.append(row[0:-1])
  label2.append(row[-1])

In [ ]:
for i,r in enumerate(label2):
  if r == 'Benign':
    label2[i] = 0
  else:
    label2[i] = 1

In [ ]:
test_data2 = np.asarray(data2)
test_label2 = tf.keras.utils.to_categorical(label2, 2)

In [ ]:
test_loss2, test_accuracy2 = model.evaluate(test_data2, test_label2)

4725/4725 [==============================] - 9s 2ms/step - loss: 1.2302 - accuracy: 0.5000


In [ ]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss2, test_accuracy2*100.))

Test loss: 1.23. Test accuracy: 50.00%
